In [4]:
# Install dependencies (run once per environment)
%pip install -q dspy pandas python-dotenv

# dataset comes from here: https://www.kaggle.com/datasets/nikitpatel/invoice-ner-dataset?resource=download


Note: you may need to restart the kernel to use updated packages.


In [5]:
# Basic imports and environment setup
import os
import json
import dspy
import pandas as pd
from dotenv import load_dotenv

# Load API keys from .env (OPENAI_API_KEY is expected)
load_dotenv()

# Configure DSPy default LM similar to other notebooks
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)
dspy.configure(lm=lm)

print("DSPy configured for invoice extraction.")


DSPy configured for invoice extraction.


In [ ]:
# Load dataset and preview a few rows
# Prefer absolute path next to this notebook; fallback to project root
nb_dir = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.path.abspath("./dspy")
candidates = [
    os.path.abspath("invoice_ner_clean.csv"),
]
for p in candidates:
    if os.path.exists(p):
        csv_path = p
        break
else:
    raise FileNotFoundError("invoice_ner_clean.csv not found in expected locations")

raw_df = pd.read_csv(csv_path)
print("Rows:", len(raw_df))
print(raw_df.head(2))

# Parse Final_Output JSON strings into dicts
raw_df["Final_Output"] = raw_df["Final_Output"].apply(lambda s: json.loads(s))

# Build small train/test splits for quick iteration
# Keep it tiny for a simple example; adjust as needed
train_df = raw_df.iloc[:30].copy()
valid_df = raw_df.iloc[30:40].copy()

def to_examples(df):
    examples = []
    for _, row in df.iterrows():
        examples.append(dspy.Example(text=row["Input"], target=row["Final_Output"]).with_inputs("text"))
    return examples

train_examples = to_examples(train_df)
valid_examples = to_examples(valid_df)

print(f"Train examples: {len(train_examples)}, Valid examples: {len(valid_examples)}")


Rows: 39
                                               Input  \
0  Beige Elegant Professional Business Invoice\n\...   
1  Black and White Clean Modern Invoice\n\nConsul...   

                                        Final_Output  
0  {"invoice_number":"1234","invoice_date":"2030-...  
1  {"invoice_number":"INV-01234","invoice_date":"...  
Train examples: 30, Valid examples: 9


In [11]:
# Define a signature for extracting a JSON dict of invoice fields
class InvoiceExtraction(dspy.Signature):
    """
    Extract key-value invoice fields as a JSON dict from free-form invoice text.
    """
    text: str = dspy.InputField(description="Raw invoice text")
    rationale: str = dspy.OutputField(description="Brief reasoning, list detected fields")
    extracted: dict = dspy.OutputField(description="JSON dict with dataset keys and string values")

# Base module (simple Predict)
extractor = dspy.Predict(InvoiceExtraction)

print("Signature and extractor ready.")


Signature and extractor ready.


In [12]:
# Define a per-field accuracy metric: correct_fields / total_gold_fields
from dspy.evaluate import Evaluate


def normalize_dict(d: dict) -> dict:
    if d is None:
        return {}
    def norm_key(k: str) -> str:
        # Uppercase keys to align with dataset convention
        return str(k).strip().upper()
    def norm_val(v: str) -> str:
        s = str(v).strip()
        s = s.replace(",", "")
        s = s.replace("$ ", "$")
        s = " ".join(s.split())  # collapse internal whitespace
        return s
    return {norm_key(k): norm_val(v) for k, v in d.items()}


def field_accuracy_metric(example: dspy.Example, pred: dspy.Prediction, trace=None, pred_name=None, pred_trace=None) -> float:
    """
    Per-field accuracy metric with optional GEPA feedback.
    Returns a float for normal evaluation; when GEPA passes pred_name/pred_trace,
    returns dspy.Prediction(score=..., feedback=...).
    """
    gold = normalize_dict(example.target)
    got = normalize_dict(getattr(pred, "extracted", {}))
    if not gold:
        return 0.0 if pred_name is None else dspy.Prediction(score=0.0, feedback="No gold fields present.")

    total = len(gold)
    correct_keys = []
    mismatched = {}
    for k, v in gold.items():
        if k in got and got[k] == v:
            correct_keys.append(k)
        else:
            mismatched[k] = (v, got.get(k, "MISSING"))
    extra_keys = [k for k in got.keys() if k not in gold]

    score = len(correct_keys) / total

    if pred_name is None:
        return score

    # Build concise feedback for GEPA to refine prompts toward dataset schema
    lines = [
        f"Matched {len(correct_keys)}/{total} fields.",
    ]
    if mismatched:
        missed_list = ", ".join(list(mismatched.keys())[:8])
        lines.append(f"Missing/mismatched: {missed_list}.")
    if extra_keys:
        extra_list = ", ".join(extra_keys[:8])
        lines.append(f"Extra keys not in schema: {extra_list}.")

    feedback = "\n".join(lines)
    return dspy.Prediction(score=score, feedback=feedback)


# Provide the devset at construction per latest API
evaluate = Evaluate(devset=valid_examples, metric=field_accuracy_metric, ordered=True)

# Standard usage: pass the module directly
initial_score = evaluate(extractor)
print("Initial field accuracy on valid:", initial_score)


2025/09/16 22:00:28 INFO dspy.evaluate.evaluate: Average Metric: 1.125 / 9 (12.5%)


Initial field accuracy on valid: EvaluationResult(score=12.5, results=<list of 9 results>)


In [13]:
# Compare base vs optimized predictions on a sample
sample = valid_examples[0]
print("Sample comparison:")
print("\nINPUT TEXT:")
print("-" * 80)
print(sample.text[:300], "...")
print("-" * 80)

print("\nBASE MODEL PREDICTION:")
base_pred = extractor(text=sample.text)
print(f"Rationale: {base_pred.rationale}")
print("\nExtracted fields:")
for k,v in (base_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nGROUND TRUTH:")
print("\nExtracted fields:")
for k,v in sample.target.items():
    print(f"{k:20s}: {v}")


Sample comparison:

INPUT TEXT:
--------------------------------------------------------------------------------
Yellow and Black Professional Company Invoice

No Items Qty Price Total
1 Cement 50 $20.00 $1000.00
2 Pvc Pipe 10 $10.00 $100.00
3 Brick 10 $10.00 $100.00
4 Wood Board 10 $10.00 $100.00
Taylor Alonso
Ginyard International Bank
Bank Code (123-456-7890)
Total $1200.00
INVOICE
Ingoude
Company
Jonathan  ...
--------------------------------------------------------------------------------

BASE MODEL PREDICTION:
Rationale: Parsed the invoice text and extracted identifiable fields. Detected:
- Vendor: "Yellow and Black Professional Company"
- Invoice number: "#123456789"
- Invoice date: "3rd January 2024" (ISO: 2024-01-03)
- Line items (4): Cement, Pvc Pipe, Brick, Wood Board with qty, unit price, line totals
- Calculated subtotal from line items: $1300.00
- Total shown on invoice: $1200.00 (mismatch vs calculated subtotal)
- Bank: "Ginyard International Bank" and "Bank Code (123-4

In [14]:
# Optimize with GEPA (similar to comedian-agent)
from dspy import GEPA

optimizer = GEPA(
    metric=field_accuracy_metric,
    auto="light",
    num_threads=8,
    track_stats=True,
    use_merge=False,
    reflection_lm=dspy.LM(model='gpt-5', temperature=1.0, max_tokens=32000),
)

optimized_program = optimizer.compile(
    extractor,
    trainset=train_examples,
    valset=valid_examples,
)

# Standard usage: evaluate the optimized program directly
opt_score = evaluate(optimized_program)
print("Optimized field accuracy on valid:", opt_score)


2025/09/16 22:01:44 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 416 metric calls of the program. This amounts to 10.67 full evals on the train+val set.
2025/09/16 22:01:44 INFO dspy.teleprompt.gepa.gepa: Using 9 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/416 [00:00<?, ?rollouts/s]2025/09/16 22:01:44 INFO dspy.evaluate.evaluate: Average Metric: 1.125 / 9 (12.5%)
2025/09/16 22:01:44 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.125
2025/09/16 22:01:44 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.125


Average Metric: 0.62 / 3 (20.8%): 100%|██████████| 3/3 [00:39<00:00, 13.14s/it]

2025/09/16 22:02:24 INFO dspy.evaluate.evaluate: Average Metric: 0.625 / 3 (20.8%)


2025/09/16 22:05:06 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for self: You are given free-form invoice text. Extract exactly the following eight fields and return a single JSON object (no extra keys, no explanations, no surrounding text):

- INVOICE_NUMBER
- INVOICE_DATE
- BILLED_TO
- COMPANY
- TOTAL_AMOUNT
- BANK_NAME
- ACCOUNT_NAME
- ACCOUNT_NUMBER

General requirements:
- Output must be a flat JSON dictionary with exactly those eight uppercase keys, in any order.
- If a field cannot be found, include the key with an empty string "" as its value.
- Do not include any fields other than the eight listed above.
- Preserve values as they appear in the source (e.g., keep currency symbols like $ and punctuation such as hyphens in account numbers). Normalize only minor spacing (e.g., remove extra internal spaces like "TOTAL $ 346.50" -> "$346.50").
- Do not reformat dates; use the date as written in the invoice.
- Do not compute totals; use the explicitly labeled final

Average Metric: 1.38 / 3 (45.8%): 100%|██████████| 3/3 [00:24<00:00,  8.19s/it]

...

2025/09/17 00:58:17 INFO dspy.teleprompt.gepa.gepa: Iteration 45: Proposed new text for self: Task: Extract exactly eight fields from free-form invoice text and return a single JSON object.

Fields (uppercase keys):
- INVOICE_NUMBER
- INVOICE_DATE
- BILLED_TO
- COMPANY
- TOTAL_AMOUNT
- BANK_NAME
- ACCOUNT_NAME
- ACCOUNT_NUMBER

Output format:
- Return exactly one valid JSON object with these eight uppercase keys. Use double quotes for JSON keys and string values. No extra text.
- If a field cannot be found, include the key with an empty string "".
- Preserve values exactly as they appear in the source (keep currency symbols, commas, periods, hyphens, and internal spaces), except:
  - Normalize minor spacing in currency values (e.g., "$ 346.50" -> "$346.50").
  - For INVOICE_NUMBER, strip a single leading "#" if present (keep internal hyphens/punctuation like "INV-00001").
- Do not reformat dates; use them exactly as written.
- Do not compute totals; use only the explicitly labeled fina

Average Metric: 1.62 / 3 (54.2%): 100%|██████████| 3/3 [00:52<00:00, 17.49s/it] 

2025/09/17 00:59:57 INFO dspy.evaluate.evaluate: Average Metric: 1.625 / 3 (54.2%)


2025/09/17 01:00:50 INFO dspy.teleprompt.gepa.gepa: Iteration 46: Proposed new text for self: You will receive free-form invoice text. Extract exactly eight fields and return a single JSON object (no extra keys, no explanations, no surrounding text):

- INVOICE_NUMBER
- INVOICE_DATE
- BILLED_TO
- COMPANY
- TOTAL_AMOUNT
- BANK_NAME
- ACCOUNT_NAME
- ACCOUNT_NUMBER

Strict output format:
- Return exactly one valid JSON object with these eight uppercase keys. Use double quotes for all JSON keys and string values.
- No extra text before/after the JSON.
- If a field cannot be found, include the key with an empty string "".
- Preserve values exactly as they appear in the source (keep currency symbols, commas, periods, hyphens, and internal spaces), except:
  - Normalize minor spacing in currency values (e.g., "$ 346.50" -> "$346.50").
  - For INVOICE_NUMBER, strip a leading "#" if present (keep internal hyphens, e.g., "INV-00001").
- Do not reformat dates; use them exactly as written.
- Do no

Optimized field accuracy on valid: EvaluationResult(score=43.06, results=<list of 9 results>)


In [ ]:
# Evaluate the optimized program on validation set
opt_score = evaluate(optimized_program)
print("Optimized field accuracy on validation:", opt_score)


2025/09/16 17:08:22 INFO dspy.evaluate.evaluate: Average Metric: 3.228846153846154 / 10 (32.3%)


Optimized field accuracy on validation: EvaluationResult(score=32.29, results=<list of 10 results>)


In [16]:
optimized_program.save("./invoice_program/", save_program=True)

In [ ]:
# Quick demo on a random validation example
sample = valid_examples[0]
print("Sample comparison:")
print("\nINPUT TEXT:")
print("-" * 80)
print(sample.text[:300], "...")
print("-" * 80)

print("\nBASE MODEL PREDICTION:")
base_pred = extractor(text=sample.text)
print(f"Rationale: {base_pred.rationale}")
print("\nExtracted fields:")
for k,v in (base_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nOPTIMIZED MODEL PREDICTION:")
opt_pred = optimized_program(text=sample.text)
print(f"Rationale: {opt_pred.rationale}")
print("\nExtracted fields:")
for k,v in (opt_pred.extracted or {}).items():
    print(f"{k:20s}: {v}")

print("\nGROUND TRUTH:")
print("\nExtracted fields:")
for k,v in sample.target.items():
    print(f"{k:20s}: {v}")


Sample comparison:

INPUT TEXT:
--------------------------------------------------------------------------------
Green Blue Pink Vintage Retro Freelance Invoice

NO. DESCRIPTION RATE QTY TOTAL
01 Graphic design for website 45 20 900.00
02 Graphic design for social content 45 8 360.00
03 Account management fee 280 1 280.00
04 Two days onsite 640 2 1280.00
SUB TOTAL $ 2820.00
TAX $ 225.60
S&H nil
TOTAL $ 3045.60 ...
--------------------------------------------------------------------------------

BASE MODEL PREDICTION:
Rationale: Detected invoice fields parsed from the text:
- Invoice title
- Invoice number ("NO. 001")
- Invoice date ("02.05.2024")
- From (biller) name, company, email, address, code
- To (client) name, company, email, address, code
- Line items (4 entries) with rate, quantity, totals
- Subtotal, Tax, Shipping (S&H), Total
- Bank / account details (Account Name, Bank, Acc)
- Payment instructions / terms ("Please pay within 60 days", cheques payable)
- Notes text

Extracte

In [ ]:
# Compare predictions with ground truth
print("\nField Comparison:")
print("-" * 80)
for field in sample.target.keys():
    gt_value = sample.target[field]
    opt_value = opt_pred.extracted.get(field, "MISSING")
    match = "✓" if gt_value == opt_value else "✗"
    print(f"{field:20s}: {match}")
    if match == "✗":
        print(f"  Ground truth: {gt_value}")
        print(f"  Prediction:   {opt_value}")
print("-" * 80)



Field Comparison:
--------------------------------------------------------------------------------
INVOICE_NUMBER      : ✓
DATE                : ✗
  Ground truth: 02.05.2024
  Prediction:   MISSING
BILL_TO             : ✗
  Ground truth: Rosa Maria Aguado
  Prediction:   MISSING
COMPANY             : ✗
  Ground truth: Aldenaire & Partners
  Prediction:   MISSING
ADDRESS             : ✗
  Ground truth: 123 Anywhere St., Any City, ST 12345
  Prediction:   MISSING
EMAIL               : ✗
  Ground truth: hello@reallygreatsite.com
  Prediction:   MISSING
BANK_NAME           : ✓
BANK_ACCOUNT        : ✓
ITEM_DESCRIPTION    : ✗
  Ground truth: Two days onsite
  Prediction:   MISSING
QTY                 : ✗
  Ground truth: 2
  Prediction:   MISSING
RATE                : ✗
  Ground truth: 640
  Prediction:   MISSING
AMOUNT              : ✗
  Ground truth: 1280.00
  Prediction:   MISSING
SUBTOTAL            : ✗
  Ground truth: $ 2820.00
  Prediction:   2820.00
TAX                 : ✗
  Ground tr

In [15]:
# Show the last interaction for transparency
print("\nInspecting last prompt:")
optimized_program(text=sample.text)
dspy.inspect_history(n=1)


Inspecting last prompt:




[2025-09-17T07:57:38.671773]

System message:

Your input fields are:
1. `text` (str): Raw invoice text
Your output fields are:
1. `rationale` (str): Brief reasoning, list detected fields
2. `extracted` (dict): JSON dict with dataset keys and string values
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## rationale ## ]]
{rationale}

[[ ## extracted ## ]]
{extracted}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You will receive free-form invoice text. Extract exactly eight fields and return a single JSON object (no extra keys, no explanations, no surrounding text):
        
        - INVOICE_NUMBER
        - INVOICE_DATE
        - BILLED_TO
        - COMPANY
        - TOTAL_AMOUNT
        - BANK_NAME
        - ACCOUNT_NAME
    